In [9]:
import requests
import json
from IPython.display import display, HTML, clear_output
from ipywidgets import Button, Textarea, VBox, HBox, Layout, Output
import time

BASE = "http://localhost:5557"
out = Output()

def health(b=None):
    with out:
        clear_output(wait=True)
        try:
            r = requests.get(f"{BASE}/api/health", timeout=5)
            data = r.json()
            color = "lime" if data.get("success") else "red"
            display(HTML(f"<h2 style='color:{color}'>FAITHH HEALTH: {data.get('status', 'DOWN')}</h2>"))
            display(HTML(f"<pre style='background:#111; color:#0f0; padding:15px; border-left:4px solid #0f0'>{json.dumps(data, indent=2)}</pre>"))
        except Exception as e:
            display(HTML(f"<h2 style='color:red'>SERVER DOWN: {str(e)}</h2>"))

def chat(b=None):
    msg = chat_input.value.strip()
    if not msg:
        return
    with out:
        clear_output(wait=True)
        display(HTML(f"<p><b>You:</b> {msg}</p>"))
        try:
            r = requests.post(f"{BASE}/api/chat", json={"message": msg}, timeout=10)
            resp = r.json().get("response", "No response")
            display(HTML(f"<p style='color:#0f0'><b>FAITHH:</b> {resp}</p>"))
        except Exception as e:
            display(HTML(f"<p style='color:red'>Error: {e}</p>"))

def rag_search(b=None):
    q = rag_input.value.strip()
    if not q:
        return
    with out:
        clear_output(wait=True)
        display(HTML(f"<h3>Searching for: <i>{q}</i></h3>"))
        try:
            r = requests.post(f"{BASE}/api/rag_search", json={"query": q}, timeout=10)
            results = r.json().get("results", [])
            display(HTML(f"<b>Found {len(results)} results:</b>"))
            for i, doc in enumerate(results[:3], 1):
                display(HTML(f"<details><summary>Result {i} ({len(doc)} chars)</summary><p style='background:#111; padding:10px; border-left:4px solid #0f0; margin:5px; white-space:pre-wrap'>{doc[:800]}...</p></details>"))
        except Exception as e:
            display(HTML(f"<p style='color:red'>RAG Error: {e}</p>"))

# Layout with buttons and inputs
health_btn = Button(description="CHECK HEALTH", button_style='success', layout=Layout(width='200px'))
chat_btn = Button(description="SEND CHAT", button_style='info', layout=Layout(width='200px'))
rag_btn = Button(description="RAG SEARCH", button_style='primary', layout=Layout(width='200px'))

health_btn.on_click(health)
chat_btn.on_click(chat)
rag_btn.on_click(rag_search)

chat_input = Textarea(placeholder="Type message to FAITHH...", layout=Layout(width='100%', height='60px'))
rag_input = Textarea(placeholder="Search knowledge base...", layout=Layout(width='100%', height='60px'))

panel = VBox([
    HBox([health_btn, chat_btn, rag_btn]),
    chat_input,
    rag_input,
    out
])

display(panel)

# Auto-run health on load
health()